In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg



from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)
sa_dask = dg.from_geopandas(sa, npartitions=4)

del sa

CPU times: user 128 ms, sys: 16.2 ms, total: 144 ms
Wall time: 168 ms


### pga data

In [3]:
%%time
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))
pga_dask = dg.from_geopandas(pgaGeo, npartitions=4)

del pga
del pgaGeo


CPU times: user 3min 21s, sys: 15.7 s, total: 3min 36s
Wall time: 3min 54s


# Merge SA data with PGA events (dask-geodataframe)

In [4]:
def dask_sjoin(dd1, dd2):
    join = gpd.sjoin(dd1, dd2, op="within")
    return join

In [5]:
%%time
PGA_SA = dd.map_partitions(dask_sjoin, pga_dask, sa_dask)

CPU times: user 214 ms, sys: 105 ms, total: 319 ms
Wall time: 1.07 s


/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  


In [7]:
%%time
# if PGA_SA is dask-geodataframe then can be sent to parquet
PGA_SA.to_parquet('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_SA.parquet')

[###########                             ] | 27% Completed |  0.5s

/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  
/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  


[################                        ] | 40% Completed |  0.9s

/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  
/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  


[################################        ] | 80% Completed |  4.4s

...

[####################################    ] | 90% Completed |  6.1s

/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  
/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  


[########################################] | 100% Completed | 33min 33.5s


/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  


IsADirectoryError: [Errno 21] Is a directory: '/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_SA.parquet'

In [8]:
test = PGA_SA[PGA_SA['event_id'] == 0]

In [ ]:
test

In [10]:
test = dd.read_parquet('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_SA.parquet')

IndexError: list index out of range